In [1]:
from numpy import array,roll,concatenate,sum,prod,unique,argwhere,zeros,isin,argmin,copy,around,zeros_like
from numpy.linalg import det
from numpy.random import randint,rand
from time import perf_counter,sleep
import threading

In [2]:
N = 16
off = 18

#### Hermitian Matrix

In [3]:
M = array((range(N),range(N))).T # diagonals

In [4]:
off = randint(0,N,(off,2)) # sparse off-diagonals
print(off.T)
off = off[off[:,1]!=off[:,0]] # diagonal pruning
off = unique(off,axis=0)
M = concatenate((M,off,roll(off,1,1)))

[[13  2 13  7  8  6 15  2 14  5  7 12 10 13  7  9 12  8]
 [14 10 14 15  9  9  4 15 11  4  2  7  7  6 12 15  9 15]]


In [5]:
data = dict(zip([tuple(index) for index in M],20*rand(len(M))))

In [6]:
M = list([*data.keys()])

In [7]:
print(1-len(M)/N/N)

0.8125


In [8]:
def reconstruction(data):
    M = array([*data.keys()])
    N = 1+M.max()
    M = zeros((N,N))
    for index,value in data.items():
        M[index] = value
    return M

In [9]:
import sympy as sp
Mat = sp.Matrix(reconstruction(data))
Mat

Matrix([
[13.804217028345,              0.0,               0.0,              0.0,              0.0,              0.0,              0.0,              0.0,              0.0,               0.0,              0.0,              0.0,              0.0,              0.0,              0.0,              0.0],
[            0.0, 1.55170279906269,               0.0,              0.0,              0.0,              0.0,              0.0,              0.0,              0.0,               0.0,              0.0,              0.0,              0.0,              0.0,              0.0,              0.0],
[            0.0,              0.0,  16.5974878421949,              0.0,              0.0,              0.0,              0.0, 13.8444340315866,              0.0,               0.0, 3.34626708554062,              0.0,              0.0,              0.0,              0.0,   1.574746029891],
[            0.0,              0.0,               0.0, 13.4626746528445,              0.0,              0.0,          

In [10]:
Mat.det()

127348736701094.

In [11]:
det(reconstruction(data))

127348736701094.17

#### Functions

In [12]:
def sgn(index):
    row,col = index
    if (row+col)%2 == 0:
        return +1
    return -1

In [13]:
def sgn(choice,parent):
    if choice[1] < parent[1]: # comparing column split
        return -1
    return +1

In [14]:
def sgn(choice,parent):
    sign = +1
    if choice[1] < parent[1]: # comparing column split
        sign *= -1
    if choice[0] < parent[0]: # comparing row split
        sign *= -1
    return sign

In [15]:
def sgn(sigma):
    # cycle tracing
    N = len(sigma)
    signature = +1
    track = list(sigma.keys())
    while len(track)>0:
        count = 0
        index = track[0]
        while index in sigma and index in track:
            track.remove(index)
            index = sigma[index]
            count += 1
            
        signature *= (-1)**(count-1)
    return signature

def increment(index,sigma):
    row,col = index
    sigma[row] = col
    return sigma

In [16]:
def minor(M,index):
    row,col = index
    M = M[M[:,0]!=row]
    M = M[M[:,1]!=col]
    return M

def attacking(M):
    if len(M)==1:
        return False
    return True

def choices(M):
    index,distribution = unique(M[:,0],return_counts=True)
    mini = index[argmin(distribution)]
    return M[M[:,0]==mini]

def distributionNull(M,N):
    rows = unique(M[:,0])
    cols = unique(M[:,1])
    if N > len(rows) or N > len(cols) or N==0:
        return True
    return False

In [17]:
def product_det(M,sigma):
    value = []
    for index in M:
        value.append(data[tuple(index.tolist())])
        increment(index,sigma)
    return prod(value)*sgn(sigma)

In [18]:
def recurDet(M,N,sigma=dict()):
    if distributionNull(M,N):
        stats['terminal'] += 1
        return 0
    elif attacking(M):
        value = [data[tuple(index.tolist())]*recurDet(minor(M,index),N-1,increment(index,sigma.copy())) for index in choices(M)]
        return sum(value)
    else:
        stats['leaf'] += 1
        return product_det(M,sigma)

In [19]:
start = perf_counter()
stats={'terminal':0,'leaf':0}

print(threading.active_count())
print('*********************')
print(recurDet(copy(M),copy(N)))
print('*********************')

print(threading.active_count())

end = perf_counter()
print('Time:',end-start)
print(stats)

6
*********************
127348736701095.06
*********************
6
Time: 0.19050370000000005
{'terminal': 84, 'leaf': 1304}


### Characteristic Polynomial

In [20]:
coeffs = zeros(N+1)
coeffs[0] = 1

In [21]:
def characteristicEdit(index,m,coeffs):
    if index[0]==index[1]: # diagonal
        return m*coeffs - roll(coeffs,1)
    return m*coeffs

In [22]:
def product_poly(coeffs,M,sigma):
    values = []
    for index in M:
        m = data[tuple(index.tolist())]
        values.append(characteristicEdit(index,m,coeffs))
        increment(index,sigma)
    return sum(values,axis=0)*sgn(sigma)

In [23]:
def charPoly(coeffs,M,N,sigma=dict()):
    if distributionNull(M,N):
        stats['terminal'] += 1
        return zeros_like(coeffs)    
    elif attacking(M):
        values = []
        for index in choices(M):
            m = data[tuple(index.tolist())]
            coefficients = characteristicEdit(index,m,coeffs.copy())
            values.append(charPoly(coefficients,minor(M,index),N-1,increment(index,sigma.copy())))
        return sum(values,axis=0)
    else:
        stats['leaf'] += 1
        return product_poly(coeffs,M,sigma.copy())

In [24]:
start = perf_counter()
print(threading.active_count())
stats={'terminal':0,'leaf':0}

poly = charPoly(coeffs,copy(M),copy(N))
print(poly)
print(threading.active_count())

end = perf_counter()
print('Time:',end-start)
print(stats)

6
> c:\users\chisti\appdata\local\temp\ipykernel_15856\3864744341.py(8)product_poly()

ipdb> len(values)
1
ipdb> values
[array([ 4.00608641e+14, -1.04933272e+15,  1.21011023e+15, -8.21628864e+14,
        3.70135489e+14, -1.18008826e+14,  2.76835452e+13, -4.89478109e+12,
        6.61537322e+11, -6.87692408e+10,  5.49377443e+09, -3.34420365e+08,
        1.52333543e+07, -5.02729997e+05,  1.13508442e+04, -1.56833912e+02,
        1.00000000e+00])]
ipdb> l
      3     for index in M:
      4         m = data[tuple(index.tolist())]
      5         values.append(characteristicEdit(index,m,coeffs))
      6         increment(index,sigma)
      7     import ipdb;ipdb.set_trace()
----> 8     return sum(values,axis=0)*sgn(sigma)

ipdb> sum(values)
15270664468969.594
ipdb> sum(values,axis=0)
array([ 4.00608641e+14, -1.04933272e+15,  1.21011023e+15, -8.21628864e+14,
        3.70135489e+14, -1.18008826e+14,  2.76835452e+13, -4.89478109e+12,
        6.61537322e+11, -6.87692408e+10,  5.49377443e+09, -3.

BdbQuit: 

In [ ]:
Mat.charpoly()

In [ ]:
poly.dtype